In [6]:
import pandas as pd 
df = pd.read_excel("Əsas data.xlsx", sheet_name="Əsas data")

# tarix sutununun deqiqlesdir
df["Order Date"] = pd.to_datetime(df["Order Date"])

# analiz gunu sec
current_day = df["Order Date"].max() + pd.Timedelta(days=1)

# RFM dataframe
rfm_df = df.groupby("Customer ID").agg({
    "Order Date" : lambda  x: (current_day- x.max()).days, # recency
    "Order ID" : "nunique",  # frequency
    "Sales" : "sum"     # monetary
}).reset_index()

rfm_df.columns = ["CustomerID", "Recency", "Frequency", "Monetary"]

rfm_df["R_Score"] = pd.qcut(rfm_df["Recency"], 5, labels=[5,4,3,2,1]).astype(int)
rfm_df["F_Score"] = pd.qcut(rfm_df["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5]).astype(int)
rfm_df["M_Score"] = pd.qcut(rfm_df["Monetary"], 5, labels=[1,2,3,4,5]).astype(int)

rfm_df["RFM_Score"] = (
    rfm_df["R_Score"].astype(str) +
    rfm_df["F_Score"].astype(str) + 
    rfm_df["M_Score"].astype(str)
)

def segments(row):
    if row["R_Score"] >= 4 and row["F_Score"] >= 4 and row["M_Score"] >= 4:
        return "VIP"
    elif row["R_Score"] >= 4:
        return "Recent"
    elif row["F_Score"] >= 4:
        return "Freguent"
    elif row["M_Score"] >= 4:
        return "High Value"
    elif row["R_Score"] >= 2 and row["F_Score"] >= 2 and row["M_Score"] >= 2:
        return "At Risk"
    else:
        return "Others"
    
rfm_df["Segment"] = rfm_df.apply(segments, axis= 1)

rfm_df.head(5)



,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,AA-103151,9,5,1445.406,5,3,3,533,Recent
1,AA-103152,14,7,6105.627,5,4,5,545,VIP
2,AA-103153,609,2,632.820,1,1,2,112,Others
3,AA-103154,185,5,5563.560,2,3,5,235,High Value
4,AA-103751,7,4,2407.245,5,2,3,523,Recent


In [ ]:
# KOR (KEY OPERATIONAL RESULTS):
# 🔹 1. VIP’ler toplam ciroya ne kadar katkı sağlıyor?

vip_ciro = rfm_df[rfm_df["Segment"] == "VIP"]["Monetary"].sum()
toplam_ciro = rfm_df["Monetary"].sum()
vip_oran = round((vip_ciro/toplam_ciro) * 100, 2)

print(f"VIP müşteriler toplam cironun %{vip_oran} kadarını sağlıyor.")

VIP müşteriler toplam cironun %34.4 kadarını sağlıyor.


In [8]:
# 🔹 2. At Risk segmentinde kaç müşteri var, ne kadar ciro riski var?
at_risk = rfm_df[rfm_df['Segment'] == 'At Risk']
at_risk_sayi = at_risk.shape[0]
at_risk_ciro = at_risk['Monetary'].sum()
print(f"At Risk segmentinde {at_risk_sayi} müşteri var. Potansiyel kayıp: ₼{at_risk_ciro:.2f}")


At Risk segmentinde 501 müşteri var. Potansiyel kayıp: ₼691163.73


In [ ]:
# 🔹 3. Segment bazlı ortalama harcama
ort_harcama = rfm_df.groupby('Segment')['Monetary'].mean().sort_values(ascending=False)
print("Segment başına ortalama harcama:")
print(ort_harcama)


In [ ]:
# 🔹 4. Sık alışveriş yapan ama düşük harcayanlar
ortalama_m = rfm_df['Monetary'].mean()
frequent_dusuk = rfm_df[(rfm_df['Segment'] == 'Frequent') & (rfm_df['Monetary'] < ortalama_m)]
print(f"Sık alışveriş yapıp az harcayan müşteri sayısı: {frequent_dusuk.shape[0]}")


In [ ]:
# 🔹 5. Segment bazlı müşteri dağılımı
print("Segment dağılımı:")
print(rfm_df['Segment'].value_counts())
